In [1]:
# from utils import *
from model import *
from dataPre import *

# from visualization.attention_visualization import createHTML
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as data_utils

import os,sys
import json
from rdkit import Chem
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
# import re
import math
import string
import warnings
warnings.filterwarnings("ignore")
# embeddings = None
from tqdm import tqdm

torch.cuda.set_device(4)

kpcb
8835
fabp4
2797


In [2]:
def train(attention_model,train_loader,criterion,optimizer,epochs = 5,use_regularization = False,C=0,clip=False):
    """
        Args:
            attention_model : {object} model
            train_loader    : {DataLoader} training data loaded into a dataloader
            optimizer       :  optimizer
            criterion       :  loss function. Must be BCELoss for binary_classification and NLLLoss for multiclass
            epochs          : {int} number of epochs
            use_regularizer : {bool} use penalization or not
            C               : {int} penalization coeff
            clip            : {bool} use gradient clipping or not
       
        Returns:
            accuracy and losses of the model
        """
    losses = []
    accs = []
    for i in range(epochs):
        print("Running EPOCH",i+1)
        total_loss = 0
        n_batches = 0
        correct = 0
        for batch_idx,(lines, contactmap,properties) in enumerate(train_loader):  
            input, seq_lengths, y = make_variables(lines, properties,smiles_letters)
            attention_model.hidden_state = attention_model.init_hidden()
            contactmap = create_variable(contactmap)
            y_pred,att = attention_model(input,contactmap)
            #penalization AAT - I
            if use_regularization:
                attT = att.transpose(1,2)
                identity = torch.eye(att.size(1))
                identity = Variable(identity.unsqueeze(0).expand(train_loader.batch_size,att.size(1),att.size(1))).cuda()
                penal = attention_model.l2_matrix_norm(att@attT - identity)
            if not bool(attention_model.type) :
                #binary classification
                #Adding a very small value to prevent BCELoss from outputting NaN's
                correct+=torch.eq(torch.round(y_pred.type(torch.DoubleTensor).squeeze(1)),y.type(torch.DoubleTensor)).data.sum()
                if use_regularization:
                    loss = criterion(y_pred.type(torch.DoubleTensor).squeeze(1),y.type(torch.DoubleTensor))+(C * penal.cpu()/train_loader.batch_size)
                else:
                    loss = criterion(y_pred.type(torch.DoubleTensor).squeeze(1),y.type(torch.DoubleTensor))
            total_loss+=loss.data
            optimizer.zero_grad()
            loss.backward() #retain_graph=True
           
            #gradient clipping
            if clip:
                torch.nn.utils.clip_grad_norm(attention_model.parameters(),0.5)
            optimizer.step()
            n_batches+=1
            if batch_idx %20000==0: 
                print(batch_idx)
                
        avg_loss = total_loss/n_batches
        acc = correct.numpy()/(len(train_loader.dataset))
        
        losses.append(avg_loss)
        accs.append(acc)
        
        print("avg_loss is",avg_loss)
        print("train ACC = ",acc)
        
        torch.save(attention_model.state_dict(), './model_pkl/DUDE/hhDUDE30Res-fold1-%d.pkl'%(i+1))
    return losses,accs

In [3]:
def getROCE(predList,targetList,roceRate):
    p = sum(targetList)
    n = len(targetList) - p
    predList = [[index,x] for x in enumerate(predList)]
    predList = sorted(predList,key = lambda x:x[1])
    tp1 = 0
    fp1 = 0
    maxIndexs = []
    for x in predList:
        if(targetList[x[0]] == 1):
            tp1 += 1
        else:
            fp1 += 1
            if(fp1>((roceRate*n)/100)):
                break
    roce = (tp1*n)/(p*fp1)
    return roce

In [8]:
def test(attention_model,test_loader,criterion,use_regularization = False):
    """
        Args:
            attention_model : {object} model
            train_loader    : {DataLoader} training data loaded into a dataloader
            optimizer       :  optimizer
            criterion       :  loss function. Must be BCELoss for binary_classification and NLLLoss for multiclass
            epochs          : {int} number of epochs
            use_regularizer : {bool} use penalization or not
            C               : {int} penalization coeff
            clip            : {bool} use gradient clipping or not
        Returns:
            accuracy and losses of the model
        """
    losses = []
    accuracy = []
    print('test begin ...')
    total_loss = 0
    n_batches = 0
    correct = 0
    all_pred = np.array([])
    all_target = np.array([])
    with torch.no_grad():
        for batch_idx,(lines, contactmap,properties) in enumerate(test_loader):
            input, seq_lengths, y = make_variables(lines, properties,smiles_letters)
            attention_model.hidden_state = attention_model.init_hidden()
            contactmap = contactmap.cuda()
            y_pred,att = attention_model(input,contactmap)
            if not bool(attention_model.type) :
                #binary classification
                #Adding a very small value to prevent BCELoss from outputting NaN's
                pred = torch.round(y_pred.type(torch.DoubleTensor).squeeze(1))
                correct+=torch.eq(torch.round(y_pred.type(torch.DoubleTensor).squeeze(1)),y.type(torch.DoubleTensor)).data.sum()
                all_pred=np.concatenate((all_pred,y_pred.data.cpu().squeeze(1).numpy()),axis = 0)
                all_target = np.concatenate((all_target,y.data.cpu().numpy()),axis = 0)
                if use_regularization:
                    loss = criterion(y_pred.type(torch.DoubleTensor).squeeze(1),y.type(torch.DoubleTensor))+(C * penal.cpu()/train_loader.batch_size)
                else:
                    loss = criterion(y_pred.type(torch.DoubleTensor).squeeze(1),y.type(torch.DoubleTensor))
            total_loss+=loss.data
            n_batches+=1
    testSize = round(len(test_loader.dataset),3)
    testAcc = round(correct.numpy()/(n_batches*test_loader.batch_size),3)
    testRecall = round(metrics.recall_score(all_target,np.round(all_pred)),3)
    testPrecision = round(metrics.precision_score(all_target,np.round(all_pred)),3)
    testAuc = round(metrics.roc_auc_score(all_target, all_pred),3)
    testLoss = round(total_loss/n_batches,5)
    print("test size =",testSize,"  test acc =",testAcc,"  test recall =",testRecall,"  test precision =",testPrecision,"  test auc =",testAuc,"  test loss = ",testLoss)
    roce1 = round(getROCE(all_pred,all_target,0.5),2)
    roce2 = round(getROCE(all_pred,all_target,1),2)
    roce3 = round(getROCE(all_pred,all_target,2),2)
    roce4 = round(getROCE(all_pred,all_target,5),2)
    print("roce0.5 =",roce1,"  roce1.0 =",roce2,"  roce2.0 =",roce3,"  roce5.0 =",roce4)
    return testAcc,testRecall,testPrecision,testAuc,testLoss,all_pred,all_target,roce1,roce2,roce3,roce4
 

In [5]:
def binary_classfication(attention_model,train_loader,epochs=20,use_regularization=True,C=1.0,clip=True):
    attention_model.cuda()
    loss = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(attention_model.parameters(),lr=0.0007)
    losses,accs = train(attention_model,train_loader,loss,optimizer,epochs,use_regularization,C,clip)
    return losses,accs

In [6]:
LSTM_HID_DIM = 64
D_A = 32
R_HEAD = 10
BATCH_SIZE = 1
EPOCHS = 50

attention_model = DrugVQA(batch_size=BATCH_SIZE,lstm_hid_dim=LSTM_HID_DIM,d_a = D_A,r=R_HEAD,block = ResidualBlock,
                          n_chars_smi = N_CHARS_SMI,n_chars_seq = N_CHARS_SEQ,num_classes=[2, 2, 2, 2],n_classes=1)
losses,accs = binary_classfication(attention_model,train_loader=train_loader,epochs=EPOCHS,use_regularization=False,C=0.03,clip=True)

Running EPOCH 1
0


KeyboardInterrupt: 

In [ ]:
LSTM_HID_DIM = 64
D_A = 32
R_HEAD = 10
BATCH_SIZE = 1
EPOCHS = 50
attention_model = DrugVQA(batch_size=BATCH_SIZE,lstm_hid_dim=LSTM_HID_DIM,d_a = D_A,r=R_HEAD,block = ResidualBlock,
                          n_chars_smi = N_CHARS_SMI,n_chars_seq = N_CHARS_SEQ,num_classes=[2, 2, 2, 2],n_classes=1)
attention_model.cuda()
testpklPath =  './model_pkl/DUDE/'
testpkl = os.listdir(testpklPath)
print(len(testpkl))
testpkl = [x for x in testpkl if 'DUDE30Res-fold1' in x and '15' in x]
testpkl = sorted(testpkl)
print(len(testpkl))
resultDict = {}
for path in testpkl:
    resultProteinDict = {}
    attention_model.load_state_dict(torch.load('./model_pkl/DUDE/'+path,map_location=lambda storage,loc:storage.cuda(4)))
    loss = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(attention_model.parameters())
    print('-------------------------------testEpoch--------------------------------',path)
    for x in testProteinList:
        print('\n current test protein:',x)
        data = dataDict[x]
        test_dataset = ProDataset(dataSet = data,seqContactDict = seqContactDict)
        test_loader = DataLoader(dataset=test_dataset,
                         batch_size=1, shuffle=True,drop_last = True)
        
        testAcc,testRecall,testPrecision,testAuc,testLoss,all_pred,all_target,roce1,roce2,roce3,roce4= test(attention_model,test_loader,loss)
        resultProteinDict[x] = [testAcc,testRecall,testPrecision,testAuc,testLoss,roce1,roce2,roce3,roce4]
    resultDict[path] = resultProteinDict

59
1
-------------------------------testEpoch-------------------------------- DUDE30Res-fold1-15.pkl

 current test protein: kpcb_2i0eA_full
test begin ...
